In [3]:
#Load Data and Sample


In [4]:
#data sample
import pandas as pd
data = pd.read_csv(r'C:\Users\Welcome\OneDrive - NSBM\Desktop\3rd_year\flight delay\flight_data_2024.csv')
data_sample = data.sample(100_000, random_state=42)

C:\Users\Welcome\AppData\Local\Temp\ipykernel_21044\551047572.py:3: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\Welcome\OneDrive - NSBM\Desktop\3rd_year\flight delay\flight_data_2024.csv')


In [7]:
from sklearn.impute import KNNImputer

# Select columns with missing values for KNN imputation
#look at 5 rows and predicting the missing value
impute_cols = ['dep_time', 'arr_time', 'actual_elapsed_time', 'air_time', 
               'taxi_out', 'taxi_in', 'carrier_delay', 'weather_delay',
               'nas_delay', 'security_delay', 'late_aircraft_delay']

imputer = KNNImputer(n_neighbors=5)
data_sample[impute_cols] = imputer.fit_transform(data_sample[impute_cols])


In [9]:
#Time-of-Day Binning
def get_time_bin(dep_time):
    if dep_time >= 500 and dep_time < 1200:
        return 'morning'
    elif dep_time >= 1200 and dep_time < 1700:
        return 'afternoon'
    elif dep_time >= 1700 and dep_time <= 2359:
        return 'evening'
    else:
        return 'night'

data_sample['dep_time_bin'] = data_sample['dep_time'].apply(get_time_bin)


In [11]:
#Airline + Origin Airport → Into one new feature
data_sample['carrier_origin'] = data_sample['op_unique_carrier'] + '_' + data_sample['origin']


In [15]:
#target encoding
#These columns are categorical and have many unique values, replace each category with a number that represents how it affects the target.

import category_encoders as ce

features = ['month', 'day_of_week', 'op_unique_carrier', 'origin',
            'dest', 'crs_dep_time', 'distance', 'dep_time_bin', 'carrier_origin']

target = (data_sample['arr_delay'] > 15).astype(int)

# Target encoding for high-cardinality categoricals
encoder = ce.TargetEncoder(cols=['op_unique_carrier', 'origin', 'dest', 'carrier_origin', 'dep_time_bin'])
X_encoded = encoder.fit_transform(data_sample[features], target)


In [16]:
import os

results_folder = r'C:\Users\Welcome\OneDrive - NSBM\Desktop\3rd_year\flight delay\flight-delay-prediction-ml\flight-delay-prediction-ml\results'
os.makedirs(results_folder, exist_ok=True)

X_encoded.to_csv(os.path.join(results_folder, 'processed_features_advanced.csv'), index=False)
target.to_csv(os.path.join(results_folder, 'target_advanced.csv'), index=False)


In [17]:
print(X_encoded.shape)


(100000, 9)


In [18]:
print(X_encoded.isnull().sum())


month                0
day_of_week          0
op_unique_carrier    0
origin               0
dest                 0
crs_dep_time         0
distance             0
dep_time_bin         0
carrier_origin       0
dtype: int64


In [19]:
print(X_encoded[['dep_time_bin', 'carrier_origin']].head())


         dep_time_bin  carrier_origin
1996999      0.107609        0.173913
848          0.209605        0.277970
6697762      0.107609        0.135401
1810495      0.209605        0.235771
828505       0.209605        0.242802


In [20]:
print(X_encoded[['origin', 'dest', 'op_unique_carrier', 'carrier_origin']].head())


           origin      dest  op_unique_carrier  carrier_origin
1996999  0.262749  0.123903           0.187981        0.173913
848      0.260826  0.181618           0.253760        0.277970
6697762  0.144243  0.163845           0.156283        0.135401
1810495  0.202312  0.211971           0.193372        0.235771
828505   0.227875  0.163090           0.193372        0.242802
